In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from mtcnn import MTCNN
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input

import os
import re

from keras.layers import Embedding
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.models import Model

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel


In [10]:
df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df = df.sample(frac=0.02,ignore_index=True)


In [4]:
face_model = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet") #include_top=True for predictions, False for embeddings


In [5]:
detector = MTCNN()

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0 
    return img

def extract_face_embeddings(frame):
    global face_embedding
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)

        return preprocessed_face

        # face_embedding = face_model.predict(preprocessed_face)      # uncomment for predictions
        # face_embeddings.append(np.squeeze(face_embedding))
        
        # return face_embeddings

# Read video
folder_path = 'MELD.Raw/dev_splits_complete/'
one_face_videos = {}

for idx, row in df.iterrows():
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_capture = cv2.VideoCapture(video_path)

        single_video_embeddings = []  # List to store embeddings for all frames

        frame_counter = 0

        while frame_counter < 2:
            ret, frame = video_capture.read()
            if not ret:
                break

            # check for more than 1 face
            if len(detector.detect_faces(frame)) == 1:

                # extract face embeddings from each frame
                extracted_embeddings = np.squeeze(extract_face_embeddings(frame))

                single_video_embeddings.append(extracted_embeddings)  # Append embeddings for this frame

                # bounding boxes
                for face in detector.detect_faces(frame):
                    x, y, w, h = face['box']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                cv2.imshow('Video', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                frame_counter += 1

            else:
                continue

        video_capture.release()
        cv2.destroyAllWindows()

        if single_video_embeddings != []:            
            one_face_videos[file_name] = np.squeeze(single_video_embeddings)
        else:
            one_face_videos[file_name] = 'too many faces'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━

In [6]:
one_face_videos.values()

dict_values([array([[[[0.36078431, 0.28235294, 0.43529412],
         [0.36078431, 0.28235294, 0.43529412],
         [0.35686275, 0.27843137, 0.43137255],
         ...,
         [0.12156863, 0.05098039, 0.03921569],
         [0.12156863, 0.05098039, 0.03921569],
         [0.12156863, 0.05098039, 0.03921569]],

        [[0.36078431, 0.28235294, 0.43529412],
         [0.36078431, 0.28235294, 0.43529412],
         [0.35686275, 0.27843137, 0.43137255],
         ...,
         [0.10980392, 0.03921569, 0.02745098],
         [0.10980392, 0.03921569, 0.02745098],
         [0.10980392, 0.03921569, 0.02745098]],

        [[0.36078431, 0.28235294, 0.43529412],
         [0.36078431, 0.27843137, 0.43529412],
         [0.35686275, 0.27843137, 0.43137255],
         ...,
         [0.09411765, 0.02352941, 0.01176471],
         [0.09411765, 0.02352941, 0.01176471],
         [0.09411765, 0.02352941, 0.01176471]],

        ...,

        [[0.41176471, 0.32156863, 0.43529412],
         [0.41176471, 0.32156863

In [13]:
df['many_faces'] = one_face_videos.values()
df['wrong_shape'] = [tf.constant(value).shape for value in one_face_videos.values()]

df = df[(df.many_faces != 'too many faces') & (df.wrong_shape == (2,224,224,3))]


c:\Python311\Lib\site-packages\pandas\core\ops\array_ops.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [15]:
ohe = OneHotEncoder(categories=[df.Emotion.unique()])
labels = ohe.fit_transform(df.Emotion.to_numpy().reshape(-1,1)).toarray()

In [25]:
#this includes xfer learning for vgg16. Switch the top layers to something appropriate. it works tho

for layer in face_model.layers:
    layer.trainable=True

base = face_model.output

flat = Flatten(name="flatten")(base)
fc = (Dense(256, activation='relu'))(flat)
fc_two = (Dense(64, activation='relu'))(fc)
output = (Dense(len(df.Emotion.unique()), activation='softmax'))(fc_two)
vid_pred_model = Model(inputs=face_model.input,outputs=output)

vid_pred_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

x = []
for video in [value for value in one_face_videos.values() if tf.constant(value).shape == (2,224,224,3)]:
    video = tf.reduce_mean(video,axis=0)
    x.append(video)

x = np.array(x)

vid_pred_model.fit(x,labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.3333 - loss: 1.6290


In [26]:
def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]

C:\Users\gsevr\AppData\Local\Temp\ipykernel_16212\1250047633.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])


In [32]:
#THIS GETS PREDICTIONS

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
token_type_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='token_type_ids')
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

bert_outputs = model(
    input_ids=input_ids,
    token_type_ids=token_type_ids,
    attention_mask=attention_mask
)

pooled_output = bert_outputs.pooler_output
dense_layer = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
output_layer = tf.keras.layers.Dense(7, activation='softmax')(dense_layer)


text_pred_model = tf.keras.Model(
    inputs=[input_ids, token_type_ids, attention_mask],
    outputs=output_layer
)


# Prepare dataset for text classification
train_texts = list(df.prepro.values)
train_labels = list(labels)

# Tokenize input texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train_labels)

train_inputs = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_inputs,
    train_labels
)).shuffle(len(train_inputs)).batch(16,drop_remainder=True)  # Adjust batch size as needed

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# Compile the model
text_pred_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fine-tune the model
text_pred_model.fit(train_dataset, epochs=3, verbose=1)  # Adjust epochs as needed


TypeError: 'NoneType' object is not callable

In [ ]:
# THIS GETS EMBEDDINGS

# Fine-tune the model
text_pred_model.fit(train_dataset, epochs=3, verbose=1)  # Adjust epochs as needed

# Extract embeddings from the model
embeddings_model = tf.keras.Model(inputs=text_pred_model.input, outputs=bert_outputs.last_hidden_state)

# Prepare data for inference
test_texts = ["Your test sentences here"]
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Convert inputs to tensors
test_inputs = {
    'input_ids': np.array(test_encodings['input_ids']),
    'token_type_ids': np.array(test_encodings['token_type_ids']),
    'attention_mask': np.array(test_encodings['attention_mask'])
}

# Get embeddings for test data
embeddings = embeddings_model.predict(test_inputs)

# 'embeddings' now contains the embeddings for the test data
print(embeddings)
